## Start OpenRefine service on MyBinder

In [ ]:
import subprocess
import time
subprocess.call("./start_refine.sh",shell=True)

#wait for 10 seconds
time.sleep(10)

In [ ]:
from google.refine import refine
import pandas as pd

## Check OpenRefine Server connection

Check connection and create an object for open refine server transaction: refine_server

In [ ]:
refine.RefineServer().get_version()

In [ ]:
refine_server = refine.Refine(refine.RefineServer())

In [ ]:
def list_project(refine_server):
    return pd.DataFrame(refine_server.list_projects()).transpose().reset_index().rename({"index":"ProjectId"},axis=1)

In [ ]:
def search_project(refine_server,project_name):
    t = list_project(refine_server)
    if t.shape[0]>0:
        t = t[t.name==project_name]
    return t

In [ ]:
search_project(refine_server,"drugs com test")

## List OpenRefine Projects using list_projects() method

In [ ]:
class RefineServerHelper():
    def __init__(self, refine_server):
        self.refine_server = refine_server
            
    def list_projects(self):
        """
        list OpenRefine project 
        """
        return pd.DataFrame(self.refine_server.list_projects()).transpose().reset_index().rename({"index":"ProjectId"},axis=1)

    def search_projects(self,project_name):
        """
        search project by name
        """
        t = self.list_projects()
        if t.shape[0]>0:
            t = t[t.name==project_name]
        #return pd.DataFrame(list(zip(t.index,t.name)),columns=["Project ID","Project Name"])        
        return t    
    
    def delete_project_byname(self,project_name):
        """
        delete project by name
        """
        t = self.search_projects(project_name)
        for x in t.ProjectId.values:
            if self.refine_server.open_project(project_id=x).delete():
                print("Project {} deleted".format(x))
                
    def open_project_byname(self,project_name,project_id=None):
        """
        open an openrefine project by name
        """
        if project_id == None:
            t = self.search_projects(project_name)
            if t.shape[0] == 0:
                raise Exception("No instance with Project Name: '{}' exist".format(project_name,t.loc[:,["ProjectId","Project Name"]]))
            if t.shape[0] > 1:
                raise Exception("More than one instance with Project Name: '{}'\n{}\nPlease specify the project id".format(project_name,t.loc[:,["ProjectId","Project Name"]]))
            project_id = t.ProjectId.values[0]
        else:
            project_name = refine_server.get_project_name(project_id)
        self.active_project = self.refine_server.open_project(project_id)
        print("OpenRefine Project {} opened, access using active_project property".format(project_name))
        return self.active_project
    
    def get_number_columns(self):
        return len(self.active_project.columns)
    
    def get_number_rows(self):
        return self.active_project.get_rows().total
    
    def get_rows(self,start=0,limit=10):    
        temp_rows = []
        for x in self.active_project.get_rows(start=start,limit = limit).rows.rows_response:
            temp_cells = []
            for y_t in self.active_project.get_models()["columnModel"]["columns"]:
                y = x["cells"][y_t["cellIndex"]]
                if y!=None:
                    temp_cells.append(y["v"])
                else:
                    temp_cells.append(None)
            temp_rows.append(temp_cells)
        return pd.DataFrame(temp_rows,columns=self.active_project.columns)    
    
    def to_lowercase(self,column_name):
        return self.active_project.text_transform(column=column_name,expression="value.toLowercase()")

    def to_uppercase(self,column_name):
        return self.active_project.text_transform(column=column_name,expression="value.toUppercase()")
        
    def to_titlecase(self,column_name):
        return self.active_project.text_transform(column=column_name,expression="value.toTitlecase()")
    
    def cluster(self,column_name,cluster_type="binning",function=None,params=None):
        return pd.DataFrame(drug_project.compute_clusters(column_name,cluster_type,function,params))

In [ ]:
refine_helper = RefineServerHelper(refine_server)

In [ ]:
refine_helper.list_projects()

## Create a New Project

In [ ]:
# Download file
!wget https://github.com/nikolausn/Data_Cleaning_Python_Exercise/blob/master/drugsComTest_raw.tsv?raw=true -O drugsComTest_raw.tsv

In [ ]:
# delete project if exist
refine_helper.delete_project_byname("drugs com test")

In [ ]:
drug_project = refine_server.new_project(project_file="drugsComTest_raw.tsv",project_name="drugs com test",separator="\t")

In [ ]:
refine_helper.search_projects("drugs com test")

In [ ]:
drug_project.list_history()

## Open the drug project

In [ ]:
drug_project = refine_helper.open_project_byname("drugs com test")

In [ ]:
refine_helper.active_project.project_name()

In [ ]:
# number of columns
refine_helper.get_number_columns()

In [ ]:
refine_helper.get_number_rows()

### Get List of Columns

In [ ]:
drug_project.columns

### Look at the rows

In [ ]:
refine_helper.get_rows(start=0,limit=20)

### create a new column for cleaning

In [ ]:
drug_project.add_column(column="review",new_column="review_cleaned")

In [ ]:
refine_helper.get_rows()

### tolower Operation on review_cleaned

In [ ]:
refine_helper.to_lowercase("review_cleaned")

In [ ]:
refine_helper.get_rows()

### Delete the first and last quote " in the review_cleaned column using regular expression

In [ ]:
# beginning quote
drug_project.text_transform(column="review_cleaned",expression="value.replace(/^\"/,\"\")")

In [ ]:
drug_project.text_transform(column="review_cleaned",expression='value.replace(/"$/,"")')

In [ ]:
refine_helper.get_rows()

## To date operation

In [ ]:
# copy column date
drug_project.add_column(column="date",new_column="date_cleaned")

In [ ]:
refine_helper.get_rows()

In [ ]:
drug_project.text_transform(column="date_cleaned",expression="value.toDate()")

In [ ]:
refine_helper.get_rows()

### Cluster operation

In [ ]:
review_cluster = drug_project.compute_clusters("review_cleaned")

In [ ]:
pd.DataFrame(review_cluster)

## merge cluster 1

In [ ]:
cluster_1 = [ x["value"] for x in review_cluster[0]]
cluster_1

In [ ]:
drug_project.mass_edit("review_cleaned",edits=[{'from':cluster_1,'to':"love it"}])

### use refine helper to show new cluster

In [ ]:
refine_helper.cluster("review_cleaned")

### Play with History

In [ ]:
drug_project.list_history()

In [ ]:
drug_project.add_column(column="review",new_column="review_test_history")

In [ ]:
refine_helper.get_rows()

In [ ]:
drug_project.list_history()

In [ ]:
drug_project.undo_project(history_id=drug_project.list_history()["past"][-2]["id"])

In [ ]:
drug_project.list_history()

In [ ]:
drug_project.add_column(column="review",new_column="review_test_history_new")

In [ ]:
drug_project.list_history()

In [ ]:
refine_helper.get_rows()

In [ ]:
!pip list